In [ ]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
									style={'textAlign': 'center', 'color': '#503D36',
										   'font-size': 40}),
										   
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites

                                dcc.Dropdown(id='site-dropdown', 
                                                options=[
                                                        {'label': 'All Sites', 'value': 'ALL'},
                                                        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                                        ],
                                                value='ALL',
                                                placeholder='Select a Launch Site here',
                                                searchable=True,
                                                style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
								
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                    min=0, max=10000, step=1000,
                                                    marks={
                                                        0: '0',
                                                        2500: '2500',
                                                        5000: '5000',
                                                        7500: '7500',
                                                        10000: '10000'
                                                        },
                                                    value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
								html.Div(dcc.Graph(id='success-payload-scatter-chart')),
								])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):

    if entered_site == 'ALL':
        line_data = spacex_df.groupby('Launch Site')['class'].mean().reset_index()
        fig = go.Figure(data=[go.Pie(labels=line_data['Launch Site'], values=line_data['class'])])
        fig.update_layout(title='Total Success Launches By Site')
        return fig

    else:
        df = spacex_df[spacex_df['Launch Site'] == entered_site]
        line_data = df.groupby('class')['Flight Number'].size().reset_index()
        fig = go.Figure(data=[go.Pie(labels=line_data['class'], values=line_data['Flight Number'])])
        fig.update_layout(title='Total Success Launches By Site')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [
                   Input(component_id='site-dropdown', component_property='value'),
                   Input(component_id='payload-slider', component_property='value')
                ]
            )

def get_pie_chart(entered_site, entered_payload):

    temp = spacex_df

    # Set the limiter
    data = temp[temp['Payload Mass (kg)'] >= entered_payload[0]]
    data = data[data['Payload Mass (kg)'] <= entered_payload[1]]

    if entered_site == 'ALL':
        scatter_data = data[['Payload Mass (kg)','class','Booster Version Category']]
        fig = px.scatter(scatter_data, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        fig.update_layout(title='Correlation between Payload and Success for all sites', xaxis_title='Payload Mass (kg)', yaxis_title='class')
        return fig

    else:
        df = data[data['Launch Site'] == entered_site]
        scatter_data = df[['Payload Mass (kg)','class','Booster Version Category']]
        fig = px.scatter(scatter_data, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        fig.update_layout(title='Correlation between Payload and Success for all sites', xaxis_title='Payload Mass (kg)', yaxis_title='class')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
